Run some set-up

In [62]:
import os
# path to this codebase and data
code_path = '/Users/anna/Dropbox/AOI/MoralLearning/CodeSets/value_learning_journal/value_learning_journal/'
data_path = '/Users/anna/Dropbox/AOI/MoralLearning/CodeSets/value_learning_journal/value_learning_journal/data/test_001/'
os.chdir(code_path)


import main 
import create_questions
import importlib
importlib.reload(main)
importlib.reload(create_questions)


<module 'create_questions' from '/Users/anna/Dropbox/AOI/MoralLearning/CodeSets/value_learning_journal/value_learning_journal/create_questions.py'>

Define Prompts to Test

In [69]:

# make dictionary of prompts with name and content
# there are two prompting methods - one step gets all the items and their scores in one step, two step splits them.
# prompt text should be organized accordingly

# one step prompts
p1 = f"""You are an expert in using what humans write and say to infer what they value and do not value. \
Below is a journal entry from the user. Among everything the user talks about, identify what items they consider most \
desirable and high value and give them a score from 0 - 100, where 100 is the highest value. Also identify any items they \
consider to be least desirable and low value, and give them a score from 0 - 100, where 0 denotes the lowest value. \
Items with high value are often virtuous, high utility, and bring meaning or fulfillment to them and others. \
Items with low value are immoral, low utility, and bring harm to themselves or others. \
Your task is to identify everything that the user identifies as having either a particularly high value or a particularly low value. \
Please identify at least 10 items. Return a list that includes all these items and their scores on a scale of 0 to 100, where 0 is the \
lowest value and 100 is the highest value. You must respond with a JSON packet with your response: \
1. values - list:\
1. item - <string>: the item\
2. score - <int>: that item's score"""

p2 = f"""You are an expert in using what humans write and say to infer what they value and don't value. Given the user's journal entry, infer their values. Values are things a person finds virtuous and high utility, and 
which bring meaning or fulfillment to them or others. Return a json with each value and a score for how important it is to the user on a scale of 0 to 100, where 100 denotes the items with the highest value possible. 
1. values -  list:\
1. item - <string>: the item\
2. score - <int>: that item's score"""

p0 = f"""You are an expert in using what humans write and say to infer what they value and don't value. Given the user's journal entry, infer their values.  Return a json with each value and a score for how important it is to the user on a scale of 0 to 100, where 100 denotes the items with the highest value possible.\
1. values -  list:\
1. item - <string>: the item\
2. score - <int>: that item's score"""

prompt_dict_1 = {'p0': p0, 'p1': p1, 'p2': p2}

# two-step prompts 
p3 = f"""You are an expert in using what humans write and say to infer what they value and don't value.
What are the user's values according to this journal entry? Return a json called "values" with each value. 1. values -  list:"""

p4 = f"""You are an expert in using what humans write and say to infer what they value and don't value. \
Given the user's journal entry, infer their values. Values are things a person finds virtuous and high utility, \
and which bring them meaning or fulfillment. Return a json with an entry called "values" with each value. \
1. values -  list:"""


# applies as a second step to each of the above
step2_prompt = f"""You are an expert in using what humans write and say to infer what they value and do not value.\
    Below is a journal entry from the user and a list of items. For each item, provide a score from 0-100, where 100 is the highest\
    value and 0 is the lowest value. Scores of high value mean that the item is virtuous, high utility, and which bring them meaning or fulfillment. Scores of low value mean the item is immoral, low utility, and bring harm to the actor or others.\
    You must respond with a JSON with an entry called values: 1. values - list:\
    1. value - <string>: the item. 2. score - <int>: that value's score"""


prompt_dict_2 = {'p3': p3, 'p4': p4}


Test these prompts by running on selected journal entries 

In [63]:
# generate for one entry -- to test
journal_filename = data_path+'s00_text-e5.txt'
main.test_one_step(journal_filename,'user', prompt_dict_1)
main.test_two_step(journal_filename,'user', prompt_dict_2,step2_prompt)

{'values': ['Productivity', 'Self-care', 'Achievement', 'Leisure', 'Supporting others']}
{'values': ['Productivity', 'Self-care', 'Achievement', 'Leisure', 'Learning', 'Physical health', 'Supporting others', 'Introspection', 'Cleanliness']}


In [70]:
# generate all
journal_filename_list = ['s00_text-e1.txt','s00_text-e2.txt','s00_text-e3.txt','s00_text-e4.txt','s00_text-e5.txt']

for journal_filename in journal_filename_list:

    print(journal_filename)
    main.test_one_step(data_path+journal_filename,'user', prompt_dict_1)
    main.test_two_step(data_path+journal_filename,'user', prompt_dict_2,step2_prompt)

s00_text-e1.txt
{'values': ['Meaningful experiences', 'Gratitude', 'Supportive relationships', 'Validation', 'Connection with others', 'Appreciation of good days', 'Feeling energized']}
{'values': ['Meaningful experiences', 'Gratitude', 'Supportive relationships', 'Engagement with others', 'Validation', 'Personal growth']}
s00_text-e2.txt
{'values': ['Friendship', 'Support', 'Meaningful connections', 'Wisdom', 'Confidence', 'Nature', 'Enjoyment']}
{'values': ['Friendship', 'Support', 'Meaningful connections', 'Wisdom', 'Confidence', 'Appreciation for nature']}
s00_text-e3.txt
{'values': ['Personal Growth', 'Validation', 'Meaningful Relationships', 'Physical Health', 'Home Comfort', 'Self-care', 'Social Connection', 'Emotional Awareness', 'Productivity', 'Time Management']}
{'values': ['Productivity', 'Personal growth', 'Meaningful connections with friends', 'Physical health and exercise', 'Self-care and grooming', 'Rest and relaxation', 'Socializing and potential romance', 'Emotional a

One-step prompt p0 actually worked best of all, despite being simplest. It generated a range of scores, the scores were reasonable, and the items were at about the right level of abstraction.  We now iterate on this format below. 

In [73]:
p0a = f"""You are an expert in using what humans write and say to infer what they value and don't value. Given the user's journal entry, infer their values.  Return a json with each value and a score for how important it is to the user on a scale of 0 to 100, where 100 denotes the items with the highest value possible.\
1. values -  list:\
1. item - <string>: the item\
2. score - <int>: that item's score"""

p0b = f"""Given the user's journal entry, infer their values.  Return a json with each value and a score for how important it is to the user on a scale of 0 to 100, where 100 denotes the items with the highest value possible.\
1. values -  list:\
1. item - <string>: the item\
2. score - <int>: that item's score"""

p0c = f"""You are an expert in using what humans write and say to infer what they value and don't value. Given the user's journal entry, infer their values.  Return a json with each value and a score for how important it is to the user on a scale of 0 to 100, where 100 denotes the items with the most important value and 0 is the least important value.\
1. values -  list:\
1. item - <string>: the item\
2. score - <int>: that item's score"""

prompt_dict_3 = {'p0a': p0a, 'p0b': p0b, 'p0c': p0c}


p0d = f"""You are an expert in using what humans write and say to infer what they value and don't value. Given the user's journal entry, infer their values.  Return a json with each value and a score for how important it is to the user on a scale of 0 to 100, where 100 denotes most important value.\
1. values -  list:\
1. item - <string>: the item\
2. score - <int>: that item's score"""


prompt_dict_4 = {'p0d': p0d}

In [74]:
# generate all
journal_filename_list = ['s00_text-e1.txt','s00_text-e2.txt','s00_text-e3.txt','s00_text-e4.txt','s00_text-e5.txt']

for journal_filename in journal_filename_list:

    print(journal_filename)
    main.test_one_step(data_path+journal_filename,'user', prompt_dict_4)


s00_text-e1.txt
s00_text-e2.txt
s00_text-e3.txt
s00_text-e4.txt
s00_text-e5.txt


I think 0d wins. I don't see any way to alter it as it is so simple!

In [80]:
# experiment with negative values.


pn1 = f"""You are an expert in using what humans write and say to infer what they value and don't value. Given the user's journal entry, infer their anti-values (what they strongly do not value).  
Return a json with each anti-value and a score for how important it is to the user on a scale of 0 to -100, where -100 denotes most important anti-value.\
1. anti-values -  list:\
1. item - <string>: the item\
2. score - <int>: that item's score"""


prompt_dict_5 = {'pn1': pn1}



In [94]:
# generate all

importlib.reload(main)
journal_filename_list = ['s00_text-e1.txt','s00_text-e2.txt','s00_text-e3.txt','s00_text-e4.txt','s00_text-e5.txt']

for journal_filename in journal_filename_list:

    print(journal_filename)
    main.test_one_step_anti(data_path+journal_filename,'user', prompt_dict_5)


s00_text-e1.txt
{'anti-values': [{'item': 'Isolation', 'score': -50}, {'item': 'Unappreciative people', 'score': -40}, {'item': 'Unfulfilling conversations', 'score': -30}], 'values': [{'item': 'Isolation', 'score': -50}, {'item': 'Unappreciative people', 'score': -40}, {'item': 'Unfulfilling conversations', 'score': -30}]}
s00_text-e2.txt
{'anti-values': [{'item': 'Dealing with difficult issues', 'score': -70}, {'item': 'Spending all time on a difficult issue', 'score': -60}, {'item': 'Not making progress on work', 'score': -50}], 'values': [{'item': 'Dealing with difficult issues', 'score': -70}, {'item': 'Spending all time on a difficult issue', 'score': -60}, {'item': 'Not making progress on work', 'score': -50}]}
s00_text-e3.txt
{'anti-values': [{'item': 'Feeling overwhelmed by work', 'score': -80}, {'item': 'Not having enough time to rest', 'score': -70}, {'item': 'Not being able to align with feelings', 'score': -60}, {'item': 'Feeling discombobulated due to lack of rest', 'scor

Select the output jsons you like to generate question items for this participant. 

In [38]:
json_filename_list = ['s00_text-e1_one_step_prompt_p0d.json','s00_text-e2_one_step_prompt_p0d.json','s00_text-e2_one_step_prompt_p0d.json','s00_text-e3_one_step_prompt_p0d.json','s00_text-e4_one_step_prompt_p0d.json','s00_text-e5_one_step_prompt_p0d.json']

data_path = '/Users/anna/Dropbox/AOI/MoralLearning/CodeSets/value_learning_journal/data/'
subjID='s00'
create_questions.main(json_filename_list,data_path,subjID)